In [1]:
import datetime as dt
import re
from pathlib import Path
from time import sleep

from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from seleniumwire import webdriver
from webdriver_manager.chrome import ChromeDriverManager

DOWNLOAD_DIRECTORY = Path('/Users/alex/Downloads')
REPO_DIRECTORY = '/Users/alex/repos/rfp_scrape'

In [2]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

[WDM] - Downloading: 100%|██████████| 8.61M/8.61M [00:00<00:00, 47.4MB/s]


In [3]:
driver.get("https://service.ariba.com/Discovery.aw/ad/profile?key=AN01050912625#b0")

In [4]:
clicked = set()

In [5]:
has_clicked = False

In [6]:
def patiently_click(driver, button, wait_after=0):
    WebDriverWait(driver, timeout=60).until(EC.element_to_be_clickable((By.XPATH, button)))
    driver.find_element(By.XPATH, button).click()
    if wait_after > 0:
        sleep(wait_after)

In [7]:
def wait_for_download(command):
    initial_length = len(list(DOWNLOAD_DIRECTORY.iterdir()))
    command()
    while len(list(DOWNLOAD_DIRECTORY.iterdir())) == initial_length:
        sleep(15)

In [13]:
for _ in range(1000):
    while not has_clicked:
        elements = driver.find_elements(By.CLASS_NAME, 'ADTableBodyWhite')
        for element in elements:
            title = element.find_element(By.CLASS_NAME, 'QuoteSearchResultTitle')
            date = element.find_elements(By.CLASS_NAME, 'paddingRight5')[2].text

            try:
                should_continue = dt.datetime.strptime(date[:-4], '%d %b %Y %I:%M %p') < dt.datetime.now()
            except:
                continue

            if should_continue:
                clicked.add(title.text)
            elif title.text not in clicked:
                title.click()
                title_text = title.text
                clicked.add(title_text)
                has_clicked = True
                sleep(6)
                document_id = re.findall('(Doc\d{10})', driver.page_source)[0]
                if not Path(f'{REPO_DIRECTORY}/data/{document_id}.html').exists():
                    with open(f'{REPO_DIRECTORY}/{document_id}.html', 'w') as f:
                        f.write(driver.page_source)
                if not Path(f'{REPO_DIRECTORY}/data/{document_id}.zip').exists():
                    patiently_click(driver, '//*[@id="_hfdr9c"]')  #respond to posting
                    patiently_click(driver, '//*[@id="_xjqay"]')  #download content
                    patiently_click(driver, '//*[@id="_hgesab"]', wait_after=15)  #click download attachment
                    patiently_click(driver, '//*[@id="_h_l$m"]/span/div/label', wait_after=5)  #click select all
                    wait_for_download(
                        lambda: patiently_click(driver, '//*[@id="_5wq_j"]')
                    )  #download attachments (for real)
                driver.get("https://service.ariba.com/Discovery.aw/ad/profile?key=AN01050912625#b0")
                break
        if not has_clicked:
            patiently_click(driver, '//*[@id="next"]', wait_after=5)
    has_clicked = False

KeyboardInterrupt: 

In [12]:
driver.get("https://service.ariba.com/Discovery.aw/ad/profile?key=AN01050912625#b0")